# View Campaign and Interactions

In the first notebook `Personalize_BuildCampaign.ipynb` you successfully built and deployed a recommendation model using deep learning with Amazon Personalize.

This notebook will expand on that and will walk you through adding the ability to react to real time behavior of users. If their intent changes while browsing a movie, you will see revised recommendations based on that behavior.

It will also showcase demo code for simulating user behavior selecting movies before the recommendations are returned.

Below we start with just importing libraries that we need to interact with Personalize

In [2]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

Below you will paste in the campaign ARN that you used in your previous notebook. Also pick a random user ID from 50 - 300. 

Lastly you will also need to find your Dataset Group ARN from the previous notebook.

In [5]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
HRNN_Campaign_ARN = "arn:aws:personalize:us-east-1:059124553121:campaign/personalize-demo-camp"

# Define User 
USER_ID = "676"

# Dataset Group Arn:
datasetGroupArn = "arn:aws:personalize:us-east-1:059124553121:dataset-group/personalize-kms"

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

## Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [6]:
response = personalize.create_event_tracker(
    name='MovieClickTracker',
    datasetGroupArn=datasetGroupArn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

arn:aws:personalize:us-east-1:059124553121:event-tracker/ce27ed68
7c315f3d-feac-40a0-a70c-08edffb02e37


## Configuring Source Data

Above you'll see your tracking ID and this has been assigned to a variable so no further action is needed by you. The lines below are going to setup the data used for recommendations so you can render the list of movies later.

In [7]:
# Interaction config
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'])
pd.set_option('display.max_rows', 5)
data = data[data['RATING'] > 3]                # keep only movies rated 3
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
data

,USER_ID,ITEM_ID,TIMESTAMP
5,298,474,884182806
7,253,465,891628467
...,...,...,...
99991,676,538,892685437
99996,716,204,879795543


In [9]:
# Item Config
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1')
items.columns = ['ITEM_ID', 'TITLE']

user_id, item_id, _ = data.sample().values[0]
item_title = items.loc[items['ITEM_ID'] == item_id].values[0][-1]
print("USER: {}".format(user_id))
print("ITEM: {}".format(item_title))

items

USER: 551
ITEM: Hard Target (1993)


,ITEM_ID,TITLE
0,2,GoldenEye (1995)
1,3,Four Rooms (1995)
...,...,...
1679,1681,You So Crazy (1994)
1680,1682,Scream of Stone (Schrei aus Stein) (1991)


## Getting Recommendations

Just like in the previous notebook it is a great idea to get a list of recommendatiosn first and then see how additional behavior by a user alters the recommendations.

In [10]:
# Get Recommendations as is
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = USER_ID,
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Raiders of the Lost Ark (1981)",
  "Silence of the Lambs, The (1991)",
  "Forrest Gump (1994)",
  "Pulp Fiction (1994)",
  "Amadeus (1984)",
  "Godfather, The (1972)",
  "Schindler's List (1993)",
  "Fugitive, The (1993)",
  "Empire Strikes Back, The (1980)",
  "Usual Suspects, The (1995)",
  "Terminator 2: Judgment Day (1991)",
  "Shawshank Redemption, The (1994)",
  "Apollo 13 (1995)",
  "Fargo (1996)",
  "Sting, The (1973)",
  "To Kill a Mockingbird (1962)",
  "Casablanca (1942)",
  "Alien (1979)",
  "Dances with Wolves (1990)",
  "Aliens (1986)",
  "When Harry Met Sally... (1989)",
  "Hoop Dreams (1994)",
  "Princess Bride, The (1987)",
  "Dead Poets Society (1989)",
  "Back to the Future (1985)"
]
[{'itemId': '174'}, {'itemId': '98'}, {'itemId': '69'}, {'itemId': '56'}, {'itemId': '191'}, {'itemId': '127'}, {'itemId': '318'}, {'itemId': '79'}, {'itemId': '172'}, {'itemId': '12'}, {'itemId': '96'}, {'itemId': '64'}, {'itemId': '28'}, {'itemId': '100'}, {'itemI

## Simulating User Behavior

The lines below provide a code sample that simulates a user interacting with a particular item, you will then get recommendations that differ from those when you started.

In [11]:
session_dict = {}

In [12]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
)

Immediately below this line will update the tracker as if the user has clicked a particular title.

In [13]:
# Pick a movie, we will use ID 207 or Gattica
send_movie_click(USER_ID=USER_ID, ITEM_ID=207)

After executing this block you will see the alterations in the recommendations now that you have event tracking enabled and that you have sent the events to the service.

In [14]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = HRNN_Campaign_ARN,
    userId = str(USER_ID),
)

item_list = get_recommendations_response['itemList']
title_list = [items.loc[items['ITEM_ID'] == np.int(item['itemId'])].values[0][-1] for item in item_list]

print("Recommendations: {}".format(json.dumps(title_list, indent=2)))
print(item_list)

Recommendations: [
  "Silence of the Lambs, The (1991)",
  "Empire Strikes Back, The (1980)",
  "Sting, The (1973)",
  "Raiders of the Lost Ark (1981)",
  "Fugitive, The (1993)",
  "Dead Poets Society (1989)",
  "Shawshank Redemption, The (1994)",
  "Apollo 13 (1995)",
  "Vertigo (1958)",
  "It's a Wonderful Life (1946)",
  "Usual Suspects, The (1995)",
  "Casablanca (1942)",
  "Forrest Gump (1994)",
  "Terminator 2: Judgment Day (1991)",
  "Pulp Fiction (1994)",
  "Amadeus (1984)",
  "Monty Python and the Holy Grail (1974)",
  "Alien (1979)",
  "2001: A Space Odyssey (1968)",
  "Dances with Wolves (1990)",
  "Blade Runner (1982)",
  "When Harry Met Sally... (1989)",
  "Graduate, The (1967)",
  "Fargo (1996)",
  "Groundhog Day (1993)"
]
[{'itemId': '98'}, {'itemId': '172'}, {'itemId': '194'}, {'itemId': '174'}, {'itemId': '79'}, {'itemId': '196'}, {'itemId': '64'}, {'itemId': '28'}, {'itemId': '479'}, {'itemId': '496'}, {'itemId': '12'}, {'itemId': '483'}, {'itemId': '69'}, {'itemId': 

## Conclusion

You can see now that recommendations are altered by changing the movie that a user interacts with, this system can be modified to any application where users are interacting with a collection of items. These tools are available at any time to pull down and start exploring what is possible with the data you have.

Finally when you are ready to remove the items from your account, open the `Cleanup.ipnb` notebook and execute the steps there.


In [65]:
eventTrackerArn = response['eventTrackerArn']
print("Tracker ARN is: " + str(eventTrackerArn))

Tracker ARN is: arn:aws:personalize:us-east-1:059124553121:event-tracker/abca43a2
